In [15]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from inv_helper import RedNet
import torch
import tqdm
import wandb
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR

from pathlib import Path
import mysql.connector as connector

In [16]:
home = os.path.expanduser('~')
os.chdir(home) # b/c we will be using universal paths

host = '127.0.0.1'
user = 'root' # change to your username
password = 'vasya1' # change to your password
database = 'ai_proj_2025' # we should all have this as the db name 

try:
    conn = connector.connect(
        host = host, 
        user = user, 
        password = password, 
        database = database
    )
    print('success')
except connector.Error as err:
    print(err)

success


In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [18]:
model = RedNet(50) # choosing a depth; the rest of the params are the default
test_data = torch.randn(2, 3, 32, 32)
model(test_data).shape


torch.Size([2, 2048, 8, 8])

In [19]:
model = nn.Sequential(model, nn.Flatten(1, -1), nn.Linear(2048*8*8, 2))


In [20]:
research_dir = Path(home, 'Desktop', 'Education', 'Spring 2025', 'AI', 'research_ai_class_spring_2025')
# research_dir = Path(home, "ai_research_proj_spring_2025", "research_ai_class_spring_2025") # in lab 409 computer for Agafia
os.chdir(research_dir)

from data_helper import SQLDataset_Informative

os.chdir(home)

In [21]:
from torchvision.transforms import v2

# transforms
transformations = v2.Compose([
    v2.RandomResizedCrop(size=(32, 32), antialias=True), 
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [22]:
# create train, val, and test sets

data_dir=Path('OneDrive - Stephen F. Austin State University', 'CrisisMMD_v2.0','CrisisMMD_v2.0')

train_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_train=True)
val_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_val=True)
test_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_test=True) 

In [23]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=64)
val_loader = DataLoader(val_set, batch_size=64)
test_loader = DataLoader(test_set, batch_size=128)

In [24]:
# validation fn

def dev(model, val_loader):
    model.to(device)
    batch_size = val_loader.batch_size
    avg = 'macro' # used when computing certain accuracy metrics
    model.eval()

    eval_loss = 0

    all_preds = []
    all_trues = []

    with torch.no_grad():
        for b, batch in tqdm.tqdm(enumerate(val_loader), 
                             total= len(val_loader), desc=f"Processing validation data"):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            raw_logits = model.forward(images)

            preds = torch.argmax(raw_logits, dim=1) # https://discuss.pytorch.org/t/cross-entropy-loss-get-predicted-class/58215

            loss = nn.CrossEntropyLoss()(raw_logits, labels)

            eval_loss += loss.item()

            all_preds.extend(preds.tolist())
            all_trues.extend(labels.tolist())


        # metrics 
        acc_total = accuracy_score(y_true=all_trues, y_pred=all_preds)
        precision = precision_score(y_true=all_trues, y_pred=all_preds, zero_division=0, average=avg)
        recall = recall_score(y_true=all_trues, y_pred=all_preds, zero_division=0, average=avg)
        f1 = f1_score(y_true=all_trues, y_pred=all_preds, zero_division=0, average=avg)

        avg_eval_loss = eval_loss / (len(val_loader))

        metrics = {
            'accuracy': acc_total, 
            'precision': precision, 
            'recall': recall, 
            'f1': f1, 
            'avg_eval_loss': avg_eval_loss
        }
        wandb.log(metrics)
        print('****Evaluation****')
        print(f'total_accuracy: {acc_total}')

        return acc_total
    


In [25]:
def train_eval(model, num_epochs, run_name, lr, architecture):
    # training hyperparameters & functions/tools
    lr = lr 
    num_epochs = num_epochs
    run_name = run_name
    

    best_val_acc = 0.0
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) #stocastic gradient descent for our optimization algorithm
    lr_sched = MultiStepLR(optimizer=optimizer, milestones=list(range(50, num_epochs, 30)), gamma=.1)

    model.to(device)
    # for saving the models
    Path(research_dir, 'models' ).mkdir(parents=True, exist_ok=True)

    # before training, set up wandb for tracking purposes
    os.environ["WANDB_API_KEY"] = "5a08d1ebbf0e86ab877a128b98be3c320301b6a0"

    run = wandb.init(
        # Set the wandb entity where your project will be logged (generally your team name).
        entity="agafiabschool-stephen-f-austin-state-university",
        # Set the wandb project where this run will be logged.
        project="Research Project for CSCI-1465",
        # Track hyperparameters and run metadata.
        config={
            "learning_rate": lr,
            "architecture": architecture,
            "dataset": "CrisisMMD",
            "epochs": num_epochs,
        }, name=run_name
    )


    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}')
        wandb.log({'epoch': epoch+1})

        loss = 0

        for b, batch in tqdm.tqdm(enumerate(train_loader), 
                            total= len(train_loader), desc=f"Processing training data in epoch {epoch+1}"):
            model.train()
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            model.zero_grad() 
            optimizer.zero_grad()

            # forward pass
            raw_logits = model.forward(images)
            # loss - can use raw logits for this function b/c it applies LogSoftmax 
            loss = nn.CrossEntropyLoss()(raw_logits, labels)
            print(f'Train Loss: {loss}')
            wandb.log({'Train Loss': loss.item()})
            wandb.log({'LR': lr_sched.get_last_lr()[0]})


            # backprop!
            loss.backward()

            optimizer.step()

            if (b+1) % 20 == 0:
                print(f'batch: {b+1} ; loss: {loss.item()}')
        
        # each epoch, run validation
        acc = dev(model=model, val_loader=val_loader)
        
        if acc > best_val_acc:
            best_val_acc = acc
            torch.save(model, Path(research_dir, 'models', f'{run_name}'))
        
        lr_sched.step()
    
    return best_val_acc

In [27]:
num_epochs = 50
lr = 10**-4
run_name = f'RedNet lr={lr}'
acc = train_eval(model, num_epochs, run_name, lr=lr, architecture='RedNet')

Epoch 1


Processing training data in epoch 1:   0%|          | 0/255 [00:00<?, ?it/s]

Train Loss: 19.06531524658203


Processing training data in epoch 1:   0%|          | 1/255 [00:04<20:56,  4.95s/it]

Train Loss: 7.284673690795898


...

Train Loss: 1.7036199569702148


Processing training data in epoch 1:   9%|▊         | 22/255 [02:09<22:13,  5.72s/it]

Train Loss: 2.1624581813812256


Processing training data in epoch 1:   9%|▉         | 23/255 [02:16<22:53,  5.92s/it]


KeyboardInterrupt: 